# Tensorflow Basics
### Lecture 1 and 2 

In [2]:
import tensorflow as tf

  ---- graph 1, graph structure ----

In [7]:
tf.reset_default_graph()
x=tf.constant(2)
y=tf.constant(3)
add_op = tf.add(x,y)
mul_op = x*y
useless = x*add_op
pow_op = tf.pow(add_op,mul_op)
with tf.Session() as sess:
    z,not_useless = sess.run([pow_op,useless])
print(z,not_useless)

15625 10


#### Specify devices 

In [16]:
tf.reset_default_graph()
with tf.device('/cpu:0'): ## you can also put on /gpu:0, /gpu:1 etc
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='b')
    c = tf.reduce_sum(tf.multiply(a, b)) ## dot product

# creates a session with log_debice_placement set to Ture, it will put
# operations on that device 
# if it is set to soft, it will auto adjust 
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

91.0


#### Multiple graphes

- with multiple graphs, you can not pass variables and values accross 
- the best practice is not to have multiple but just have disconnected graphs 
- but if you insist, here is the way to do it 

In [19]:
g1 = tf.Graph()
g2 = tf.Graph()

c0 = tf.constant(0) ## op1 is added to the 'default graph which is   #different from both g1 and g2 
## default graph 
default_graph = tf.get_default_graph() 
## graph 1 
with g1.as_default():
    c1 = tf.constant(1)
    d1 = tf.constant(1)
## graph 2 
with g2.as_default():
    c2 = tf.constant(2)
    d2 = tf.constant(2)
    res = tf.add(c2,d2)
    

## default graph nodes 
node_names = [n.name for n in tf.get_default_graph().as_graph_def().node]
print('default graph nodes:',node_names)

## g1 graph nodes 
node_names = [n.name for n in g1.as_graph_def().node]
print('graph1 nodes:',node_names)

## g2 graph nodes 
node_names = [n.name for n in g2.as_graph_def().node]
print('graph2 nodes:',node_names)

default graph nodes: ['a', 'b', 'Mul', 'Const', 'Sum', 'Const_1', 'Const_2']
graph1 nodes: ['Const', 'Const_1']
graph2 nodes: ['Const', 'Const_1', 'Add']


In [20]:
## default session 
with tf.Session() as sess:
    print(sess.run(c0))  ## if run c1, it will give you an error

with tf.Session(graph = g1) as sess1:
    print(sess1.run(c1))  ## if run c0 or c2, will give you an error

with tf.Session(graph = g2) as sess2:
    print(sess2.run(res))  ## if run c0 or c2, will give you an error

0
1
4


## Tensorflow operations 

#### Constants 

- #### tf.constant(value, dtype=None, shape=None,name='Const', verify_shape=False)

In [23]:
tf.reset_default_graph()
a = tf.constant([2, 2], name="a")  ## if shpe is None and verify_shape=False, it will not check shape
b = tf.constant([2,1],shape=[2],verify_shape=True) ## it will check tensorf shape, raise error if it doesn't match

- tf.zeros(shape,dtype=tf.float32,name=None)
- tf.zeros_like(input_tensor,dtype=None,name=None,optimize=True)
- tf.fill(dim,value,name=None)     ## fill with some values

In [27]:
inp = tf.zeros([2,3],tf.int32,name='input')
z = tf.zeros_like(inp,name='zeros')
o = tf.ones_like(inp,name='ones')
eight = tf.fill([5,5],8,name='fill_8')

with tf.Session() as sess:
    print(sess.run(eight))

[[8 8 8 8 8]
 [8 8 8 8 8]
 [8 8 8 8 8]
 [8 8 8 8 8]
 [8 8 8 8 8]]


- #### tf.linspace(start, stop, num, name=None)
- #### tf.range(start, limit=None, delta=1, dtype=None, name='range')
- #### tf.range(start, limit, delta) ==> [3, 6, 9, 12, 15]

In [28]:
tf.linspace(10.0, 13.0, 4) ## will give you [10,11,12,13]
tf.range(3,18,3) ## with start with 3, but will not include 18 
tf.range(10) ## will give [0-9]

<tf.Tensor 'range_1:0' shape=(10,) dtype=int32>

- tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
- tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None,name=None)
- tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None,name=None)
- #### tf.random_shuffle(value, seed=None, name=None)
- #### tf.random_crop(value, size, seed=None, name=None)  ## random crop images 
- tf.multinomial(logits, num_samples, seed=None, name=None)
- tf.random_gamma(shape, alpha, beta=None, dtype=tf.float32, seed=None, name=None)

In [34]:
tf.reset_default_graph()
a = tf.constant([[1,2,3],[2,2,3],[3,2,3],[4,2,3]])
b = tf.random_shuffle(a)  ## always shuffle the first dimension 

In [38]:
with tf.Session() as sess:
    print(sess.run(b))

[[2 2 3]
 [1 2 3]
 [3 2 3]
 [4 2 3]]


#### What is wrong with constant

- #### constant will be stored in graph def, this makes loading graphs expensive when constants are big 
- Only use constants for primitive types.Use variables or readers for more data that requires more memory

In [42]:
tf.reset_default_graph()
my_const = tf.constant([1.0, 2.0], name="my_const")
with tf.Session() as sess:
    print (sess.graph.as_graph_def())
# you will see value of my_const stored in the graph’s definition

node {
  name: "my_const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\200?\000\000\000@"
      }
    }
  }
}
versions {
  producer: 21
}



### Variables 

In [10]:
tf.reset_default_graph()

In [11]:
a = tf.Variable(2, name="scalar")
b = tf.Variable([2, 3], name="vector")
c = tf.Variable([[0, 1], [2, 3]], name="matrix")
W = tf.Variable(tf.zeros([784,10]),name = 'weight')

- ##### tf.Variables holde several ops: .initializer .value .assign .assign_add and more

In [12]:
## easiest way to initialize all variables 
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(a))
    
## Initialize only a subset of variables:
init_ab = tf.variables_initializer([a, b], name="init_ab")
with tf.Session() as sess:
    sess.run(init_ab)
    print(sess.run(b))

2
[2 3]


In [14]:
## assign value to variable 
assign_op = a.assign(1000)
with tf.Session() as sess:
    sess.run(assign_op)  ## must run assign op 
    print(a.eval())

1000


### Placeholder

#### tf.placeholder(dtype, shape=None, name=None)
- shape=None means that tensor of any shape will be accepted as value for placeholder.
- shape=None also breaks all following shape inference, which makes many ops not work because they expect certain rank.

In [27]:
tf.reset_default_graph()
# create a placeholder of type float 32-bit, shape is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3])
b = tf.constant([5, 5, 5], tf.float32)
c = a + b # Short for tf.add(a, b)
## see if a is a feedable tensor 
tf.Graph.is_feedable(tf.get_default_graph(),tensor=a)

True

In [28]:
## feed into the session with a dictionary 
with tf.Session() as sess:
    print (sess.run(c, {a: [1, 2, 3]}))

[ 6.  7.  8.]
